<div class="alert alert-success" style = "border-radius:10px;border-width:3px;border-color:white;font-family:Verdana,sans-serif;font-size:16px;">
<h2>Data processing for Fine-tune</h2>
This Jupyter notebook is focused on processing the original dataset, which has been extracted from manually corrected file templates. The primary task involves calculating the ratios of '1's within this dataset. Based on these calculations, we filter out items that predominantly consist of '1's, specifically dropping any items where '1's constitute more than 80% of the data. The final step in this notebook is the creation of a JSON file, meticulously formatted to facilitate the fine-tuning of our model. This process ensures that our dataset is optimally prepared for the fine-tuning phase.

In [58]:
import numpy as np
import json

Calcualte the ratio's of 1s of the origianal dataset. 

In [ ]:
# Initialize counter for 1s
total_count_1s = 0
total_samples = 0

# Summing up the count of 1s for  all components
for component in components:
    true_labels = filtered_df[component].values
    total_count_1s += np.count_nonzero(true_labels == 1)
    total_samples += len(true_labels)

# Calculating the ratio of 1s to the dataset
ratio_of_1s = total_count_1s / total_samples if total_samples != 0 else 0

# Print the ratio of 1s of the dataset
print(f"Ratio of 1s across all components: {ratio_of_1s}")

Drop all the audio files where 80% of the sampels are '1's

In [ ]:
df=filtered_df
# Calculate the ratio of '1's for each row
df['1_ratio'] = df.iloc[:, 1:].sum(axis=1) / (df.shape[1] - 1)  # Exclude 'File' column

# Remove rows where '1_ratio' > 0.8
df = df[df['1_ratio'] <= 0.8]

# Drop the '1_ratio' column if you no longer need it
df = df.drop(columns=['1_ratio'])

Recalculate the ratio of '1's for the new filter dataset

In [ ]:
# Initialize counter for 1s
total_count_1s = 0
total_samples = 0

# Summing up the count of 1s for  all components
for component in components:
    true_labels = filtered_df[component].values
    total_count_1s += np.count_nonzero(true_labels == 1)
    total_samples += len(true_labels)

# Calculating the ratio of 1s to the dataset
ratio_of_1s = total_count_1s / total_samples if total_samples != 0 else 0

# Print the ratio of 1s of the dataset
print(f"Ratio of 1s across all components: {ratio_of_1s}")


Set up all the queries, incorporating the desired output and evaluation criteria for each item, along with the transcription of the conversation.

In [28]:
def evaluate_acogidaCorporativa (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Emplea saludo y presentación corporativa según canal y emisión / recepción
* Si en el transcurso de la conversación cambia el interloculor se presenta nuevamente
* Continúa con el protocolo de presentación aunque el cliente conteste a su saludo
* Comienza la presentación en un tiempo inferior a 3 seg tras recibir la llamada
* Está preparado para la siguiente llamada. Sensación positiva. Su presentación es natural, entendible y comercial
* El cliente interrumpe la presentación del asesor para exponer directamente el argumento
* Por causas ajenas al asesor no se pueda entender la presentación
0 si cumple:
* No emplea presentación corporativa
* Saludo coloquial, falta de concentración.
* Si el transcurso de la conversación cambia el interloculor no vuelve a presentarse
* Denota cansancio, apatía en el saludo, repetitividad. Su presentación es mecánica, difícil de entender y/o poco comercial.
conversación:
{conversation}
"""

    return output_string

def evaluate_identificaciónDelCliente (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Identifica de forma correcta al cliente o interlocutor.
* Pregunta y confirma todos los datos establecidos por protocolo en la política de seguridad
* No facilitar información a no tomadores de las pólizas
* No ofrece informacion que por motivos de seguridad (LOPD) Generali no desea divulgar
0 si cumple:
* No identifica al cliente o interlocutor
* No pasa pólitica de seguridad 
* Facilita informacion a no tomador
* Ofrece información que por motivos de seguridad Generali no desea divulgar
conversación:
{conversation}
"""

    return output_string

def evaluate_despedidaCorporativa  (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Antes de finalizar la llamada pregunta al cliente si puede ofrecerle ayuda adicional con fórmulas como ¿puedo ayudarle en algo más? ¿puedo facilitarle alguna información adicional?
* Cuando se corta la llamada
0 si cumple:
* No ofrece ayuda adicional
conversación:
{conversation}
"""

    return output_string

def evaluate_ofreceAyudaAdicional (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Utiliza la despedida/s establecida para cada canal y servicio
* Su despedida es natural y entendible           
* Cuando se corta la llamada
0 si cumple:
* No utiliza la despedida establecida.
* Despedida coloquial.
* Su despedida es mecánica y difícil de entender.
* No se despide, permaneciendo en silencio al finalizar la llamada.
conversación:
{conversation}
"""

    return output_string

def evaluate_correcciónGramatical  (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Se expresa de forma gramaticalmente correcta
* No emplea dequeísmos, laísmos, leísmos,…
0 si cumple:
* Utiliza dequeísmos, laísmos, leísmos,… 
* Emplea expresiones gramaticalmente incorrectas
conversación:
{conversation}
"""

    return output_string

def evaluate_expresiónOral (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Se expresa de forma clara, precisa, ordenada, concisa y formal, adaptándose al nivel del cliente. 
* Demuestra tener fluidez verbal y facilidad para emplear el lenguaje durante el transcurso de la conversación
0 si cumple:
* Se expresade forma imprecisa y desordenada. 
* Utiliza palabras de relleno, poco claras o frases hechas. 
* Utiliza palabras o expresiones que el interlocutor no entiende. 
* No demuestra tener fluidez verbal
conversación:
{conversation}
"""

    return output_string

def evaluate_Vocabulario (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Utiliza un vocabulario rico, correcto, positivo, comercial.
* Omite muletillas, jerga, palabras negras, negativas, diminutivos, superlativos, palabras imprecisas, etc. 
* No emplea términos de procedimientos internos o tecnicísmos* Se expresa de forma clara, precisa, ordenada, concisa y formal, adaptándose al nivel del cliente. 
* Demuestra tener fluidez verbal y facilidad para emplear el lenguaje durante el transcurso de la conversación
0 si cumple:
* Su vocabulario es pobre, incorrecto, negativo o poco comercial. 
* Utiliza: Palabras negras (problema), Palabras negativas (no), Palabras dubitativas (quizás), Palabras imprecisas (quizás, más o menos...),  Muletillas, argot, coloquialismos, Frases hechas, Diminutivos (un momentito), Superlativos, Tecnicismos y  expresiones propias del departamento.
conversación:
{conversation}
"""

    return output_string

def evaluate_entonaciónModulación (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Evita la monotonía, varia el ritmo, expresividad.
* Modula y entona la voz adaptándola a cada una de las frases de la conversación
* Conserva siempre un volumen medio, ni grita ni susurra
0 si cumple:
* No modula (altos/bajos), es monocorde, cansino, apático, tiene musicalidad, etc., lo que provoca que el interlocutor tenga que realizar un esfuerzo para comprender el mensaje emitido por el asesor
* Grita o susurra, sube el volumen para enfatizar
conversación:
{conversation}
"""

    return output_string

def evaluate_correctaArticulaciónElocuciónAdecuada (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Evita la monotonía, varia el ritmo, expresividad.
* Modula y entona la voz adaptándola a cada una de las frases de la conversación
* Conserva siempre un volumen medio, ni grita ni susurra
0 si cumple:
* No modula (altos/bajos), es monocorde, cansino, apático, tiene musicalidad, etc., lo que provoca que el interlocutor tenga que realizar un esfuerzo para comprender el mensaje emitido por el asesor
* Grita o susurra, sube el volumen para enfatizar
conversación:
{conversation}
"""

    return output_string

def evaluate_sonrisaTelefónica (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Emplea la sonrisa siempre que es posible. 
* Mantiene una actitud positiva, un tono alegre.
0 si cumple:
* La situación permite sonrisa y no se percibe
{conversation}
"""

    return output_string

def evaluate_FormulasDeCortesía (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Aparecen claras muestras de deferencia y respeto por el interlocutor. 
* Hace uso de fórmulas de cortesía durante el transcurso de la conversación y al interrogar al cliente. 
* Pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestica o perjuicio al cliente
0 si cumple:
* No hace uso de fórmulas de cortesía durante el transcurso de la conversación y/o al interrogar al cliente . 
* Empleo de imperativos. 
* No pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestio o perjuicio al cliente
{conversation}
"""

    return output_string

def evaluate_noInterrumpir (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Aparecen claras muestras de deferencia y respeto por el interlocutor. 
* Hace uso de fórmulas de cortesía durante el transcurso de la conversación y al interrogar al cliente. 
* Pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestica o perjuicio al cliente
0 si cumple:
* No hace uso de fórmulas de cortesía durante el transcurso de la conversación y/o al interrogar al cliente . 
* Empleo de imperativos. 
* No pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestio o perjuicio al cliente
{conversation}
"""

    return output_string

def evaluate_Esperas (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
a) Justifica la espera:
* Se avisa al cliente para mantenerle en espera 
* Utiliza las esperas de forma correcta, sólo si es necesario y en los casos de consulta al supervisor o transferencia de la llamada. 
* Cuando tiene que transferir la llamada, informa previamente de ello y emplea el argumento correcto para informar de la misma. 
* Justifica el motivo de la espera con el argumento correcto
b) Retoma la llamada:
* Cuando tiene que dejar al cliente en espera, retoma la llamada cada 30 seg
* Emplea un argumento correcto cada vez que retoma al cliente informándole de la gestión
* Presenta disculpas por haberle mantenido en espera y retoma siguiendo el protocolo establecido (Sr. Martínez, gracias por su espera) * Cuando no se da la circunstancia de dejar al interlocutor en espera en ningún momento.
0 si cumple:
a) Justifica la espera:
* No informa al cliente para mantenerle a la espera. 
* No justifica la espera, no informa previamente de la necesidad de la msma ni justifica el motivo con un argumento correcto.
b) Retoma la llamada:
* No retoma la llamada periódicamente, cada 30 seg.
* No retoma de forma correcta, lo hace sin agradecer la espera y sin ningún argumento, exponiendo directamente la solución.
* Cuando tiene que transferir no informa y/o emplea un argumento incompleto/incorrecto para informar de la transferencia.
{conversation}
"""

    return output_string

def evaluate_controlDeLosSilencios (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Evita silencios innecesarios.
* Evita el efecto "tunel oscuro": nos quedamos sin argumento y no sabemos que contestar al cliente, mantenemos en espera mucho tiempo mientras gestionamos sin retomar y justificar la espera
0 si cumple:
* Silencios innecesarios a lo largo de la conversación.
* Se queda en silencio y sin argumentos, o se mantiene en silencio prolongado mientras gestiona sin justificar
{conversation}
"""

    return output_string

def evaluate_escuchaActiva (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Manifiesta interés y atención por lo que le plantean, se adapta a la psicología del interlocutor.
* Comprende cualquier actitud. 
* Hace sentir al interlocutor que está prestando la máxima atención a sus explicaciones. 
* Interviene evitando incómodos silencios durante el transcurso de la conversación. 
* Cada vez que practica la escucha activa lo hace de forma natural.
0 si cumple:
* No se percibe esfuerzo en la atención ni interés por las explicaciones del interlocutor. 
* No entiende las reacciones y replica con las mismas formas. 
* Se producen incómodos silencios durante el transcurso de la conversación. 
* Cada vez que practica la escucha activa a lo largo de la conversación lo hace de forma mecánica.
{conversation}
"""

    return output_string

def evaluate_necesidadDelCliente (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Entiende la posición del interlocutor, empatía absoluta
* Disponibilidad para la ayuda
* Realiza un correcto sondeo (recepción) / Exposición del mensaje (emisión)
* Hace preguntas claves para obtener la información necesaria para dirigir la llamada.
* Reformula datos aportados por el cliente
* Reconstruye y sintetiza acuerdos o datos. Lo hace a lo largo de la conversación
0 si cumple:
* Revela excesiva indiferencia y frialdad
* No se esfuerza en averiguar lo que le plantean, sólo indaga lo mínimo. 
* No ofrece distintas alternativas al tema planteado ni orienta al cliente hacia la que más se adecua a sus necesidades. 
* No expone de forma estructurada.
* No repite los acuerdos.
* No aclara las ideas difusas
{conversation}
"""

    return output_string

def evaluate_personalizaciónEnElTrato (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Se dirige a la persona como un interlocutor conocido (no es uno más). 
* Una vez identificado el interlocutor (sea o no el titular) se dirige a él utilizando el tratamiento Sr./Sra. seguido del apellido o D./Dña. seguido del nombre. En caso de que el apellido sea malsonante o difícil de pronunciar se optará por el tratamiento D/Dña. 
* Siempre que retoma una llamada o tiene que llamar la atención del interlocutor se dirige utilizando el protocolo establecidido (Sr/Sra. D/Dña). 
* Siempre utiliza el tratamiento de Usted al dirigirse al cliente, salvo que expresamente lo solicite el cliente
0 si cumple:
* Da la impresión de no saber con quien habla.
* Identificado el interlocutor no hace uso del tratamiento correspondiente. 
* Al retonar la llamada o llamar la atención del interlocutor no lo hace utilizando el tratamiento correspondiente, sino de forma incorrecta y/o coloquial
* Tutea al cliente cuando el no lo solicite.
{conversation}
"""

    return output_string

def evaluate_direccionDeLaLlamada (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Sabe dirigir la llamada
* Lleva el control de la conversación, plantea alternativas
* Adopta una actitud segura, dirigiendo la conversación con el fin de que el cliente confie plenamente en el servicio. Firmeza y seguridad en las afirmaciones
* Seguridad en las reacciones y respuestas
* Cambia la percepción negativa del cliente a través de argumentos.
0 si cumple:
* No dirige la llamada siendo el cliente el que establece las pautas durante la conversación. 
* No propone alternativas.
* Se penalizan los monólogos
* Transmite inseguridad y hace que el cliente no confie plenamente en el servicio
* Utiliza expresiones o palabras dubitativas de forma reiterada.
* Titubea dejando frases a medias de forma constante.
* No logra convencer.
* Silencios breves que muestran nervios o inseguridad.
* Consulta constantemente cada pregunta que le plantea el cliente.
* También se considera incorrecto el exceso de seguridad, es decir, cuando el asesor adopta una actitud prepotente y agresiva hacia al cliente
* Mal tratamiento de objeciones. No sabe tratar las objeciones. No las rebate o lo hace sin convicción, siendo dubitarivo, impreciso, etc. 
* Permanece en silencio ante las objecciones del cliente.
* Mantiene el tono negativo de un cliente.
{conversation}
"""

    return output_string

def evaluate_imagenDeLaEmpresa (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Aporta información, buena imagen de la compañía y de sus productos y servicios. 
* Proyecta una imagen positiva de la empresa, buscando tanto el beneficio del cliente como el de la empresa. 
* Asume las responsabilidades derivadas del posible error de cualquier compañero como si hubiera sido propio.
0 si cumple:
* Crea una mala imagen de la empresa. 
* Se muestra indiferente ante el beneficio de Generali y del cliente, proyectando una imagen negativa de la empresa.
* Se excusa en los compañeros o en otros departamentos
{conversation}
"""

    return output_string

def evaluate_empatia (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Entiende la posición del interlocutor, empatía absoluta
0 si cumple:
* Revela excesiva indiferencia y frialdad
{conversation}
"""

    return output_string

def evaluate_correctoUsoAplicacionesSistemas (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Utiliza las diferentes aplicaciones de forma correcta para su consulta o la gestión de solicitudes de los clientes.
* Incidencias en los sistemas
0 si cumple:
* No realiza un uso correcto de las aplicaciones disponibles.
* No transfiere de forma correcta (cuándo proceda). 

{conversation}
"""

    return output_string

def evaluate_conocimientoAplicaciónProtocolosProcedimientos (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Conoce los procedimientos y protocolos y los aplica de forma correcta en la llamada y/o gestión.
0 si cumple:
* No conoce o no aplica los procedimientos establecidos para cada llamada, tipo de solicitud, etc. 

{conversation}
"""

    return output_string

def evaluate_clasificaciones (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Tipifica la llamada y lo hace de forma correcta en el aplicativo. La tipificación debe ajustarse al contenido de la llamada (incluyendo observaciones de la llamada en la tipificación)
* Incidencias en los sistemas
0 si cumple:
* No tipifica o tipifica de forma incorrecta la llamada

{conversation}
"""

    return output_string

def evaluate_resolucion (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Resuelve una situación que puede concluir por si mismo evitando llamadas posteriores del cliente por el mismo motivo
* Si la situación no puede resolverse online, transfiere la llamada / gestión al departamento correcto siguiendo el protocolo marcado
* Confirma con el cliente la gestión realizada
* Hace una reformulación final resumen de todo lo gestionado en la llamada.                                            
* Cuando por incidencia técnica en los sistemas no puede resolver en la llamada
0 si cumple:
* No se esfuerza en encontrar una solución a su alcance
* No verifica la comprensión del cliente
* Actitud pasiva, indiferente, despreocupación
* No confirma con el cliente la gestión realizada
* No realiza ninguna reformulación final, resumen de los acuerdos alcanzados con el mismo

{conversation}
"""

    return output_string

Generate the JSON file for the fine-tuning process, selecting only the items that are retained in the filtered dataset.

In [54]:
def createQuery(target_file):

    try:
        # Get the responses based on the condition
        file_path = "../output/" + str(target_file) + ".txt"
        responses = df.loc[df['File'] == target_file, 'component_1':'component_24'].values.tolist()[0]

        dialogs = [
            {"Inputs": "### Instruction: " + evaluate_acogidaCorporativa(file_path) + " ### Response: La valoración de este item es " + str(responses[0])},
            {"Inputs": "### Instruction: " + evaluate_identificaciónDelCliente(file_path) + " ### Response: La valoración de este item es " + str(responses[1])},
            {"Inputs": "### Instruction: " + evaluate_despedidaCorporativa(file_path) + " ### Response: La valoración de este item es " + str(responses[2])},
            {"Inputs": "### Instruction: " + evaluate_ofreceAyudaAdicional(file_path) + " ### Response: La valoración de este item es " + str(responses[3])},
            {"Inputs": "### Instruction: " + evaluate_correcciónGramatical(file_path) + " ### Response: La valoración de este item es " + str(responses[4])},
            {"Inputs": "### Instruction: " + evaluate_expresiónOral(file_path) + " ### Response: La valoración de este item es " + str(responses[5])},
            {"Inputs": "### Instruction: " + evaluate_Vocabulario(file_path) + " ### Response: La valoración de este item es " + str(responses[6])},
            {"Inputs": "### Instruction: " + evaluate_entonaciónModulación(file_path) + " ### Response: La valoración de este item es " + str(responses[7])},
            {"Inputs": "### Instruction: " + evaluate_correctaArticulaciónElocuciónAdecuada(file_path) + " ### Response: La valoración de este item es " + str(responses[8])},
            {"Inputs": "### Instruction: " + evaluate_sonrisaTelefónica(file_path) + " ### Response: La valoración de este item es " + str(responses[9])},
            {"Inputs": "### Instruction: " + evaluate_FormulasDeCortesía(file_path) + " ### Response: La valoración de este item es " + str(responses[10])},
            {"Inputs": "### Instruction: " + evaluate_noInterrumpir(file_path) + " ### Response: La valoración de este item es " + str(responses[11])},
            {"Inputs": "### Instruction: " + evaluate_Esperas(file_path) + " ### Response: La valoración de este item es " + str(responses[12])},
            {"Inputs": "### Instruction: " + evaluate_controlDeLosSilencios(file_path) + " ### Response: La valoración de este item es " + str(responses[13])},
            {"Inputs": "### Instruction: " + evaluate_escuchaActiva(file_path) + " ### Response: La valoración de este item es " + str(responses[14])},
            {"Inputs": "### Instruction: " + evaluate_necesidadDelCliente(file_path) + " ### Response: La valoración de este item es " + str(responses[15])},
            {"Inputs": "### Instruction: " + evaluate_personalizaciónEnElTrato(file_path) + " ### Response: La valoración de este item es " + str(responses[16])},
            {"Inputs": "### Instruction: " + evaluate_direccionDeLaLlamada(file_path) + " ### Response: La valoración de este item es " + str(responses[17])},
            {"Inputs": "### Instruction: " + evaluate_imagenDeLaEmpresa(file_path) + " ### Response: La valoración de este item es " + str(responses[18])},
            {"Inputs": "### Instruction: " + evaluate_empatia(file_path) + " ### Response: La valoración de este item es " + str(responses[19])},
            {"Inputs": "### Instruction: " + evaluate_correctoUsoAplicacionesSistemas(file_path) + " ### Response: La valoración de este item es " + str(responses[20])},
            {"Inputs": "### Instruction: " + evaluate_conocimientoAplicaciónProtocolosProcedimientos(file_path) + " ### Response: La valoración de este item es " + str(responses[21])},
            {"Inputs": "### Instruction: " + evaluate_clasificaciones(file_path) + " ### Response: La valoración de este item es " + str(responses[22])},
            {"Inputs": "### Instruction: " + evaluate_resolucion(file_path) + " ### Response: La valoración de este item es " + str(responses[23])}
        ]
    except Exception as e:
        print(f"Error creating dialogs for file {file_path}: {e}")
        dialogs = []
        
    return dialogs

In [56]:
# Initialize an empty list to store all the dialogs
all_dialogs = []

# Create a list of all the 'File' IDs from your DataFrame
file_ids = df['File'].tolist()

# Iterate through the list of 'File' IDs
for file_id in file_ids:
 
    # Call the createQuery function
    dialogs = createQuery(file_id)
    
    # Extend the all_dialogs list with the dialogs from the current file
    all_dialogs.extend(dialogs)

In [59]:
# Specify the file path for the JSON file
json_file_path = "output_file.json"

# Save the 'dialogs' list as a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(all_dialogs, json_file, indent=2)